# Movie recommender
<br>
## Dateset info
This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
data_folder = os.path.join(os.path.expanduser("~"),"Documents/Git/movie_recommender","ml-latest-small")
ratings_filename = os.path.join(data_folder, "ratings.csv")

ratings = pd.read_csv(ratings_filename)
ratings["timestamp"] = pd.to_datetime(ratings["timestamp"],unit="s")
ratings[:5]

,userId,movieId,rating,timestamp
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47
2,1,6,4.0,2000-07-30 18:37:04
3,1,47,5.0,2000-07-30 19:03:35
4,1,50,5.0,2000-07-30 18:48:51


### Apriori algorithm
target: to find rules like {if users like those movies -> they will like the movie}.  

create new feature: <span style="color:blue">*favorable*</span>, assuming that users like the movie if they rated it more than 3.

In [3]:
ratings["favorable"] = ratings["rating"] > 3
ratings.sample(3)

,userId,movieId,rating,timestamp,favorable
90762,590,1732,4.0,2009-11-17 01:23:23,True
94776,599,80880,2.5,2017-06-26 22:32:29,False
48130,312,2105,3.0,2003-01-21 19:21:04,False


choose part of the data as training_dataset to reduce the search space, thus boost the speed of Apriori algorithm.

In [4]:
sub_ratings = ratings[ratings["userId"].isin(range(200))]
favorable_ratings = sub_ratings[sub_ratings["favorable"]]
favorable_reviews_by_users = dict((k, frozenset(v.values))    # store v.values in frozenset
                                 for k,v in favorable_ratings.groupby("userId")["movieId"])

In [5]:
num_favorable_by_movie = sub_ratings[["movieId", "favorable"]].groupby("movieId").sum()
num_favorable_by_movie.sort_values(by="favorable", ascending=False)[:5]

,favorable
movieId,
318,95.0
356,88.0
296,79.0
593,77.0
260,71.0


In [6]:
freq_itemsets = {}
min_sup = 50
freq_itemsets[1] = dict((frozenset((movie_id,)), 
                         row["favorable"])
                        for movie_id, row in num_favorable_by_movie.iterrows()
                        if row["favorable"] > min_sup)

In [7]:
from collections import defaultdict

def find_freq_itemsets(favorable_reviews_by_users, k_1_itemsets, min_sup):
    counts = defaultdict(int)
    for user, reviews in favorable_reviews_by_users.items():    # reviews is set
        for itemset in k_1_itemsets:
            if itemset.issubset(reviews):
                for other in reviews - itemset:
                    current_superset = itemset | frozenset((other,))  # "|" is an union symbol for sets
                    counts[current_superset] += 1
    return dict([(itemset, freq) for itemset,freq in counts.items() if freq >= min_sup])

First, check each k_1 itemset whether it is subset of each user's rated as favorable movies, if yes, then add the rest favorable movives one by one to form k itemsets, and count the frequency at the same time, choose the k itemset of which the frequency is more than min_support and store them is dictionary.   

In [8]:
import sys

for k in range(2,20):
    cur_freq_itemsets = find_freq_itemsets(favorable_reviews_by_users, freq_itemsets[k-1], min_sup)
    freq_itemsets[k] = cur_freq_itemsets
    if len(cur_freq_itemsets) == 0:
        print("Did not find any frequent itemsets of length {}".format(k))
        sys.stdout.flush()
        break
    else:
        print("I found {} frequent itemsets of length {}".format(len(cur_freq_itemsets), k))
        sys.stdout.flush()

del freq_itemsets[1]    # need length of itemsets to be >= 2 to find association rules

I found 145 frequent itemsets of length 2
I found 508 frequent itemsets of length 3
I found 989 frequent itemsets of length 4
I found 1146 frequent itemsets of length 5
I found 811 frequent itemsets of length 6
I found 338 frequent itemsets of length 7
I found 73 frequent itemsets of length 8
I found 5 frequent itemsets of length 9
Did not find any frequent itemsets of length 10


**sys.stdout.flush()**, make sure to output the result even though the program is in process since it may not output the results in time in the notebook for long iterations.

In [9]:
candidate_rules = []
for itemset_length, itemset_counts in freq_itemsets.items():
    for itemset in itemset_counts.keys():
        for conclusion in itemset:
            premise = itemset - set((conclusion,))   # set((conclusion,)), transfer it so that it can be set mutable.
            candidate_rules.append((premise, conclusion))

To recommend one movie once, so all the other in candidate rules will be set as premise.

In [10]:
def cal_confi(favorable_reviews_by_users):
    correct_counts = defaultdict(int)
    incorrect_counts = defaultdict(int)

    for user, reviews in favorable_reviews_by_users.items():
        for candidate_rule in candidate_rules:
            premise, conclusion = candidate_rule
            if premise.issubset(reviews):
                if conclusion in reviews:
                    correct_counts[candidate_rule] += 1
                else:
                    incorrect_counts[candidate_rule] += 1

    rule_confi = {candidate_rule: correct_counts[candidate_rule] 
                  / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
                 for candidate_rule in candidate_rules}
    return rule_confi

rule_confi = cal_confi(favorable_reviews_by_users)

In [11]:
movie_names_filename = os.path.join(data_folder, "movies.csv")
movie_names = pd.read_csv(movie_names_filename)
movie_names.sample(3)

,movieId,title,genres
7982,96691,Resident Evil: Retribution (2012),Action|Horror|Sci-Fi|IMAX
5832,32296,Miss Congeniality 2: Armed and Fabulous (2005),Adventure|Comedy|Crime
626,798,Daylight (1996),Action|Adventure|Drama|Thriller


In [54]:
top10_rated_movieId = num_favorable_by_movie.sort_values(by="favorable", ascending=False)[:10].index
top10_rated_movies = movie_names.loc[movie_names["movieId"].isin(top10_rated_movieId),"title"]
print(top10_rated_movies)

97                                      Braveheart (1995)
224             Star Wars: Episode IV - A New Hope (1977)
257                                   Pulp Fiction (1994)
277                      Shawshank Redemption, The (1994)
314                                   Forrest Gump (1994)
461                               Schindler's List (1993)
510                      Silence of the Lambs, The (1991)
898     Star Wars: Episode V - The Empire Strikes Back...
900     Raiders of the Lost Ark (Indiana Jones and the...
1939                                   Matrix, The (1999)
Name: title, dtype: object


In [21]:
def get_movie_name(movieId):
    return movie_names.loc[movie_names["movieId"] == movieId,"title"].values[0]

In [22]:
from operator import itemgetter

def show_rules(rule_confi):
    sorted_confi = sorted(rule_confi.items(), key=itemgetter(1), reverse=True)
    for index in range(5):
        print("Rule #{}".format(index+1))
        premise, conclusion = sorted_confi[index][0]
        premise_names = ", ".join([get_movie_name(x) for x in premise])
        conclusion_name = get_movie_name(conclusion)
        print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
        print("- confidence: {0:.3f}\n".format(rule_confi[(premise, conclusion)]))

show_rules(rule_confi)

Rule #1
Rule: If a person recommends Star Wars: Episode IV - A New Hope (1977), Seven (a.k.a. Se7en) (1995), Schindler's List (1993) they will also recommend Matrix, The (1999)
- confidence: 1.000

Rule #2
Rule: If a person recommends American Beauty (1999), Star Wars: Episode IV - A New Hope (1977), Seven (a.k.a. Se7en) (1995) they will also recommend Matrix, The (1999)
- confidence: 1.000

Rule #3
Rule: If a person recommends Forrest Gump (1994), Star Wars: Episode IV - A New Hope (1977), Seven (a.k.a. Se7en) (1995) they will also recommend Matrix, The (1999)
- confidence: 1.000

Rule #4
Rule: If a person recommends Star Wars: Episode VI - Return of the Jedi (1983), Usual Suspects, The (1995), Star Wars: Episode V - The Empire Strikes Back (1980) they will also recommend Star Wars: Episode IV - A New Hope (1977)
- confidence: 1.000

Rule #5
Rule: If a person recommends Star Wars: Episode VI - Return of the Jedi (1983), Matrix, The (1999), Star Wars: Episode V - The Empire Strikes Bac

In [23]:
test_ratings = ratings[~ratings["userId"].isin(range(200))]
test_favorable = test_ratings[test_ratings["favorable"]]
test_favorable_by_users = dict((k, frozenset(v.values)) for k,v in test_favorable.groupby("userId")["movieId"])

In [24]:
test_confi = cal_confi(test_favorable_by_users)

def show_test_rules(rule_confi, test_confi):
    sorted_confi = sorted(rule_confi.items(), key=itemgetter(1), reverse=True)
    for index in range(5):
        print("Rule #{}".format(index+1))
        premise, conclusion = sorted_confi[index][0]
        premise_names = ", ".join([get_movie_name(x) for x in premise])
        conclusion_name = get_movie_name(conclusion)
        print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
        print("- Train confidence: {0:.3f}\n".format(rule_confi[(premise, conclusion)]))
        print("- Test confidence: {0:.3f}\n".format(test_confi[(premise, conclusion)]))

show_test_rules(rule_confi, test_confi)

Rule #1
Rule: If a person recommends Star Wars: Episode IV - A New Hope (1977), Seven (a.k.a. Se7en) (1995), Schindler's List (1993) they will also recommend Matrix, The (1999)
- Train confidence: 1.000

- Test confidence: 0.833

Rule #2
Rule: If a person recommends American Beauty (1999), Star Wars: Episode IV - A New Hope (1977), Seven (a.k.a. Se7en) (1995) they will also recommend Matrix, The (1999)
- Train confidence: 1.000

- Test confidence: 0.902

Rule #3
Rule: If a person recommends Forrest Gump (1994), Star Wars: Episode IV - A New Hope (1977), Seven (a.k.a. Se7en) (1995) they will also recommend Matrix, The (1999)
- Train confidence: 1.000

- Test confidence: 0.927

Rule #4
Rule: If a person recommends Star Wars: Episode VI - Return of the Jedi (1983), Usual Suspects, The (1995), Star Wars: Episode V - The Empire Strikes Back (1980) they will also recommend Star Wars: Episode IV - A New Hope (1977)
- Train confidence: 1.000

- Test confidence: 0.955

Rule #5
Rule: If a person

<p>* Try other evaluating ways...*</p>

### Recommender system-collaborative filtering

In [25]:
movie_dict = {}
for i,v in enumerate(sorted(set(movie_names["movieId"]))):
    movie_dict[v] = i

In [26]:
print("the number of films in ratings.csv is: {}\nthe number of films in movies.csv is: {}"\
      .format(len(set(ratings["movieId"])),len(set(movie_names["movieId"]))))

the number of films in ratings.csv is: 9724
the number of films in movies.csv is: 9742


In [27]:
len(movie_dict)

9742

In [28]:
cf_ratings = ratings.copy()
cf_ratings["movieNum"] = cf_ratings["movieId"].map(movie_dict)
cf_ratings[:5]

,userId,movieId,rating,timestamp,favorable,movieNum
0,1,1,4.0,2000-07-30 18:45:03,True,0
1,1,3,4.0,2000-07-30 18:20:47,True,2
2,1,6,4.0,2000-07-30 18:37:04,True,5
3,1,47,5.0,2000-07-30 19:03:35,True,43
4,1,50,5.0,2000-07-30 18:48:51,True,46


In [29]:
m = len(movie_names)   # the num of movies is 9742
n = len(set(ratings["userId"]))   # the num of users is 610

rating_matrix = np.zeros((m,n))
map_matrix = np.zeros((m,n))
for i in range(len(cf_ratings)):
    rating_matrix[cf_ratings.loc[i,"movieNum"], cf_ratings.loc[i,"userId"]-1] = cf_ratings.loc[i,"rating"]
    map_matrix[cf_ratings.loc[i,"movieNum"], cf_ratings.loc[i,"userId"]-1] = 1

In [30]:
rating_matrix[:5,:6]

array([[4., 0., 0., 0., 4., 0.],
       [0., 0., 0., 0., 0., 4.],
       [4., 0., 0., 0., 0., 5.],
       [0., 0., 0., 0., 0., 3.],
       [0., 0., 0., 0., 0., 5.]])

In [31]:
map_matrix[:5,:6]

array([[1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]])

In [32]:
k = 50   # set the num of features to be 50
X = np.random.standard_normal((m, k))
theta = np.random.standard_normal((n, k))

X.shape, theta.shape

((9742, 50), (610, 50))

In [33]:
def serialize(X, theta):
    """serialize 2 matrix
    """
    # X (movie, feature), (9742, 10): movie features
    # theta (user, feature), (610, 10): user preference
    return np.concatenate((X.ravel(), theta.ravel()))

In [34]:
def deserialize(param, n_movie, n_user, n_features):
    """into ndarray of X(9742, 10), theta(610, 10)"""
    X = param[:n_movie * n_features].reshape(n_movie, n_features)
    theta = param[n_movie * n_features:].reshape(n_user, n_features)
    return X, theta

In [35]:
def cost_func(param, Y, R, n_features, lda=1):
    n_movie, n_user = Y.shape
    X, theta = deserialize(param, n_movie, n_user, n_features)
    inner = np.multiply(X @ theta.T - Y, R)
    
    reg_term = np.power(param, 2).sum() * (lda / 2)
    
    return np.power(inner, 2).sum() / 2 + reg_term

In [36]:
def gradient(param, Y, R, n_features, lda=1):
    n_movies, n_user = Y.shape
    X, theta = deserialize(param, n_movies, n_user, n_features)

    inner = np.multiply(X @ theta.T - Y, R)
    X_grad = inner @ theta
    theta_grad = inner.T @ X

    reg_term = lda * param

    # roll them together and return
    return serialize(X_grad, theta_grad) + reg_term

In [37]:
param = serialize(X, theta)

X_grad, theta_grad = deserialize(gradient(param, rating_matrix, map_matrix, k, 1),m, n, k)

assert X_grad.shape == X.shape
assert theta_grad.shape == theta.shape

In [38]:
cost_func(param, rating_matrix, map_matrix, k, 1)

3452165.211108999

In [39]:
rating_mean = np.repeat(rating_matrix.mean(1),n).reshape(m,n)
rating_norm = rating_matrix - rating_mean

In [40]:
import scipy.optimize as opt

In [41]:
res = opt.minimize(fun=cost_func,
                   x0=param,
                   args=(rating_norm, map_matrix, k, 1),
                   method='TNC',
                   jac=gradient)

In [51]:
cost_func(res.x, rating_matrix, map_matrix, k, 1)

25747.721621011682

In [42]:
X_trained, theta_trained = deserialize(res.x, m, n, k)
X_trained.shape, theta_trained.shape

((9742, 50), (610, 50))

In [43]:
prediction = X_trained @ theta_trained.T
pred_ratings = prediction + rating_mean

In [44]:
print(np.amax(rating_matrix), np.amax(rating_mean),np.amax(prediction),np.amax(pred_ratings))

5.0 2.301639344262295 11.456723710516174 13.758363054778469


In [45]:
def get_movie_name_movieNum():
    movieNum_name = {}
    for i in range(m):     
        movieId = list(movie_dict.keys())[list(movie_dict.values()).index(i)]
        movieNum_name[i] = get_movie_name(movieId)
    return movieNum_name

movieNum_name = get_movie_name_movieNum()

In [46]:
movieNum_name[0]

'Toy Story (1995)'

In [47]:
rated_movies_user = dict((k, frozenset(v.values))    # store v.values in frozenset
                                 for k,v in cf_ratings.groupby("userId")["movieNum"])

In [48]:
def get_top_recommendations_by_user(pred_ratings, userId, top=5):
    pred_ratings_user = np.c_[np.array(np.arange(m)).T,\
                        np.array(pred_ratings[:,userId-1]).T]   # in pred_ratings, the userId starts from 0
    ratings_user_sorted = np.argsort(pred_ratings_user[:,1])[::-1]
    top_k = ratings_user_sorted[:top]
    movies, top_pred_ratings = [movieNum_name[x] for x in pred_ratings_user[top_k,0]], pred_ratings_user[top_k,1]
    i = 1
    for mv, rt in zip(movies, top_pred_ratings):
        print("Recommend#{0}: {1}, the predicted rating is {2:.0f}".format(i, mv, rt))
        i += 1

get_top_recommendations_by_user(pred_ratings, 0, top=5)

Recommend#1: Mars Attacks! (1996), the predicted rating is 9
Recommend#2: Clerks (1994), the predicted rating is 9
Recommend#3: GoldenEye (1995), the predicted rating is 8
Recommend#4: Good Will Hunting (1997), the predicted rating is 8
Recommend#5: Dead Man Walking (1995), the predicted rating is 7


In [49]:
def get_rated_movies_user(userId):
    ratings_ = ratings.loc[ratings["userId"]==userId,["movieId","rating"]].sort_values("rating",ascending=False)[:5]
    r = 1
    for i,s in zip(ratings_["movieId"], ratings_["rating"]):
        print("Top#{}liked: {}, the rating is {}".format(r, get_movie_name(i), s))
        r += 1

In [50]:
get_rated_movies_user(1)

Top#1liked: M*A*S*H (a.k.a. MASH) (1970), the rating is 5.0
Top#2liked: Excalibur (1981), the rating is 5.0
Top#3liked: Indiana Jones and the Last Crusade (1989), the rating is 5.0
Top#4liked: Pink Floyd: The Wall (1982), the rating is 5.0
Top#5liked: From Russia with Love (1963), the rating is 5.0


In [153]:
sample_test = dict([(1,4),(21,4),(34,4),(36,4),(39,5),(50,4),(58,5),(261,4),(357,2),(597,3)])
sample_df = pd.DataFrame({"movieId":[x for x in sample_test.keys()],"rating":[v for v in sample_test.values()]})
sample_df

,movieId,rating
0,1,4
1,21,4
2,34,4
3,36,4
4,39,5
5,50,4
6,58,5
7,261,4
8,357,2
9,597,3


In [58]:
rated_movieId_user = dict((k, frozenset(v.values))    # store v.values in frozenset
                                 for k,v in ratings.groupby("userId")["movieId"])

In [147]:
def euclidean_score(sample_data, user):
    common_rated = []
    for i in sample_data.movieId:
        if i in rated_movieId_user[user]:
            common_rated.append(i)
    
    if len(common_rated) == 0:
        return 0
    
    squared_differences = []
    for mid in common_rated:
        squared_differences.append(np.square(sample_data.loc[sample_data.movieId==mid,"rating"].values[0] - \
                                            ratings.loc[(ratings.userId==user) & \
                                                        (ratings.movieId==mid),"rating"].values[0]))
    return 1 / (1 + np.sqrt(np.sum(squared_differences)))

In [154]:
euclidean_score(sample_df,5)

0.3333333333333333

In [160]:
euclidean_score(sample_df,1)

0.5

In [162]:
euclidean_score(sample_df,8)

0.20521309615767264

In [158]:
similarity_scores = defaultdict(int)
for i in range(n):
    {similarity_scores[i+1]: euclidean_score(sample_df, i+1) for i in range(n)}
similarity_scores

defaultdict(int,
            {1: 0,
             2: 0,
             3: 0,
             4: 0,
             5: 0,
             6: 0,
             7: 0,
             8: 0,
             9: 0,
             10: 0,
             11: 0,
             12: 0,
             13: 0,
             14: 0,
             15: 0,
             16: 0,
             17: 0,
             18: 0,
             19: 0,
             20: 0,
             21: 0,
             22: 0,
             23: 0,
             24: 0,
             25: 0,
             26: 0,
             27: 0,
             28: 0,
             29: 0,
             30: 0,
             31: 0,
             32: 0,
             33: 0,
             34: 0,
             35: 0,
             36: 0,
             37: 0,
             38: 0,
             39: 0,
             40: 0,
             41: 0,
             42: 0,
             43: 0,
             44: 0,
             45: 0,
             46: 0,
             47: 0,
             48: 0,
             49: 0,
            

In [159]:
sorted_similarity_scores = sorted(similarity_scores.items(),key=itemgetter(1),reverse=True)
sorted_similarity_scores[0][0]

1

In [151]:
def find_most_similar_user(sample_data):
    similarity_scores = defaultdict(int)
    for i in range(n):
        {similarity_scores[i+1]: euclidean_score(sample_data, i+1) for i in range(n)}
    sorted_similarity_scores = sorted(similarity_scores.items(),key=itemgetter(1),reverse=True)
    return sorted_similarity_scores[0][0]    

In [152]:
find_most_similar_user(sample_df)

0

In [163]:
get_top_recommendations_by_user(pred_ratings, 1, top=5)

Recommend#1: Shawshank Redemption, The (1994), the predicted rating is 9
Recommend#2: Shakespeare in Love (1998), the predicted rating is 7
Recommend#3: Departed, The (2006), the predicted rating is 7
Recommend#4: Cool Hand Luke (1967), the predicted rating is 7
Recommend#5: Godfather: Part II, The (1974), the predicted rating is 7
